# Find minting data (Moralis)

In [ ]:
"""
Update Parameters Here
"""
COLLECTION_NAME = "Quaks"
CONTRACT = "0x07bbdaf30e89ea3ecf6cadc80d6e7c4b0843c729"
CHAIN = "eth"

In [ ]:
"""
A Moralis API Key is required.
The free tier includes one and is enough to get minting data.

AVAILABLE CHAINS:
    eth, ropsten, rinkeby, goerli, kovan,
    polygon, mumbai, bsc, bsc testnet,
    avalanche, avalanche testnet, fantom
"""

import time

import pandas as pd
import requests

from honestnft_utils import config, constants


def get_moralis_transfers(
    contract_address,
    chain="eth",
    limit=100,
    cursor=None,
    continuous: bool = True,
    have: list = [],
):
    all_data = []
    all_data.extend(have)

    headers = {"Content-type": "application/json", "x-api-key": config.MORALIS_API_KEY}
    url = f"https://deep-index.moralis.io/api/v2/nft/{contract_address}/transfers"
    params = {"chain": chain, "limit": limit, "cursor": cursor}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        decode_response = response.json()
        transfers = decode_response["result"]
        all_data.extend(transfers)

        if decode_response["cursor"] is not None and continuous is True:
            return get_moralis_transfers(
                contract_address=contract_address,
                chain=chain,
                limit=limit,
                cursor=decode_response["cursor"],
                continuous=continuous,
                have=all_data,
            )
        else:
            return all_data
    else:
        print(response.text)
        print("error, sleeping then calling again")
        time.sleep(5)
        return get_moralis_transfers(
            contract_address=contract_address,
            chain=chain,
            limit=limit,
            cursor=decode_response["cursor"],
            continuous=continuous,
            have=all_data,
        )


def get_mintdata(
    COLLECTION_NAME: str,
    CONTRACT: str,
    CHAIN: str,
) -> None:

    RARITY_CSV = f"{config.RARITY_FOLDER}/{COLLECTION_NAME}_raritytools.csv"
    RARITY_DB = pd.read_csv(RARITY_CSV)

    all_data = get_moralis_transfers(contract_address=CONTRACT, chain=CHAIN)

    df = pd.json_normalize(all_data)

    # remove non minting rows
    df = df.loc[df["from_address"] == constants.MINT_ADDRESS]

    # make sure token_id is an integer
    df["token_id"] = df["token_id"].astype(int)

    # add rarity rank to minting data
    df = df.merge(RARITY_DB, left_on="token_id", right_on="TOKEN_ID")

    # discard unwanted columns
    df = df[
        [
            "transaction_hash",
            "to_address",
            "token_id",
            "from_address",
            "Rank",
            "block_timestamp",
        ]
    ]

    df.drop_duplicates(subset=["token_id"], inplace=True)

    # get matching columns names to HonestNFT csv format
    df.columns = ["txid", "to_account", "TOKEN_ID", "current_owner", "rank", "time"]

    # clean 'time' field to make it compatible with the csv produced by 'find_minting_data.ipynb'
    df["time"] = df["time"].str.replace(".000Z", "", regex=False)

    df.to_csv(f"{config.MINTING_FOLDER}/{COLLECTION_NAME}_minting.csv", index=False)


get_mintdata(COLLECTION_NAME, CONTRACT, CHAIN)